# PySpark Demo Plotly Notebook
## Demo
1. Setup Spark
2. Load Kaggle Data
3. Transform Data with Spark SQL
4. Graph Data with Plotly

_Prepared by: [Gary A. Stafford](https://twitter.com/GaryStafford)   
Associated article: https://wp.me/p1RD28-61V_

### Setup Spark
Setup Spark SparkSession

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [2]:
spark = SparkSession \
    .builder \
    .appName('pyspark_demo_app') \
    .getOrCreate()

### Load Kaggle Data
Load the Kaggle dataset from the CSV file, containing ~21K records, into a DataFrame

In [3]:
bakery_schema = StructType([
    StructField('date', StringType(), True),
    StructField('time', StringType(), True),
    StructField('transaction', IntegerType(), True),
    StructField('item', StringType(), True)
])
df1 = spark.read \
    .format('csv') \
    .option('header', 'true') \
    .load('BreadBasket_DMS.csv', schema=bakery_schema)
print('DataFrame rows: %d' % df1.count())
print('DataFrame schema: %s' % df1)
df1.show(10, False)

DataFrame rows: 21293
DataFrame schema: DataFrame[date: string, time: string, transaction: int, item: string]
+----------+--------+-----------+-------------+
|date      |time    |transaction|item         |
+----------+--------+-----------+-------------+
|2016-10-30|09:58:11|1          |Bread        |
|2016-10-30|10:05:34|2          |Scandinavian |
|2016-10-30|10:05:34|2          |Scandinavian |
|2016-10-30|10:07:57|3          |Hot chocolate|
|2016-10-30|10:07:57|3          |Jam          |
|2016-10-30|10:07:57|3          |Cookies      |
|2016-10-30|10:08:41|4          |Muffin       |
|2016-10-30|10:13:03|5          |Coffee       |
|2016-10-30|10:13:03|5          |Pastry       |
|2016-10-30|10:13:03|5          |Bread        |
+----------+--------+-----------+-------------+
only showing top 10 rows



### Transform Data with Spark SQL
Transform the DataFrame's bakery data using Spark SQL

In [4]:
df1.createOrReplaceTempView('tmp_bakery')
df2 = spark.sql("SELECT date, transaction, item " +
                       "FROM tmp_bakery " +
                       "WHERE item NOT LIKE 'NONE' " +
                       "ORDER BY transaction")
print('DataFrame rows: %d' % df2.count())
df2.show(5, False)

DataFrame rows: 20507
+----------+-----------+-------------+
|date      |transaction|item         |
+----------+-----------+-------------+
|2016-10-30|1          |Bread        |
|2016-10-30|2          |Scandinavian |
|2016-10-30|2          |Scandinavian |
|2016-10-30|3          |Hot chocolate|
|2016-10-30|3          |Jam          |
+----------+-----------+-------------+
only showing top 5 rows



In [5]:
df2.createOrReplaceTempView('bakery_table_tmp2')

df3 = spark.sql("SELECT date, count(*) as count " +
                       "FROM bakery_table_tmp2 " +
                       "WHERE date >= '2017-01-01' " +
                       "GROUP BY date " +
                       "ORDER BY date")
print('DataFrame rows: %d' % df3.count())
df3.show(5, False)

DataFrame rows: 98
+----------+-----+
|date      |count|
+----------+-----+
|2017-01-01|1    |
|2017-01-03|87   |
|2017-01-04|76   |
|2017-01-05|95   |
|2017-01-06|84   |
+----------+-----+
only showing top 5 rows



### Graph Data with Plotly
Use [Plotly](https://plot.ly/python/) to create a chart showing bakery items sold over time  
Demostrates linear fit and data smoothing:
* [Plotly Python Open Source Graphing Library](https://plot.ly/python/)
* [Smoothing in Python](https://plot.ly/python/smoothing/)
* [Linear Fit in Python](https://plot.ly/python/linear-fits/)

In [6]:
import chart_studio.tools
import chart_studio.plotly as py
import plotly.graph_objs as go
from numpy import arange
from scipy import stats, signal
import warnings
warnings.filterwarnings('ignore')

In [7]:
# *** UPDATE WITH YOUR CREDENTIALS FOR PLOTLY ***
chart_studio.tools.set_credentials_file(username='garystafford', api_key='qgrXTrMz17p8JWitRjrl')

In [8]:
df4 = df3.toPandas()

# Generated linear fit
xi = arange(0, len(df4.index))
slope, intercept, r_value, p_value, std_err = stats.linregress(xi, df4['count'])
line = slope * xi + intercept

layout = dict(title='2017 Bakery Sales',
              xaxis=dict(
                  title='Month',
                  showgrid=True,
                  zeroline=True,
                  showline=True,
                  ticks='outside',
                  tickangle=45,
                  showticklabels=True),
              yaxis=dict(
                  title='Items Sold/Day',
                  showgrid=True,
                  zeroline=True,
                  showline=True,
                  ticks='outside',
                  showticklabels=True))

trace1 = go.Bar(x=df4['date'], y=df4['count'], name='Items Sold')
trace2 = go.Scatter(x=df4['date'], y=line, mode='lines', name='Linear Fit')
trace3 = go.Scatter(x=df4['date'], y=signal.savgol_filter(df4['count'], 53, 3),
                    mode='lines', name='Savitzky-Golay')
data = [trace1, trace2, trace3]
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='jupyter-basic_bar.html')